In [1]:
import pandas as pd
import os
import ta
import numpy as np

In [2]:
import pandas as pd

def detect_order_blocks(df: pd.DataFrame, lookahead: int = 3, displacement_factor: float = 0.075):
    """
    Detect bullish and bearish order blocks in OHLCV data.
    
    Parameters:
        df : DataFrame with columns ['open','high','low','close']
        lookahead : int, how many candles ahead to confirm displacement
        displacement_factor : float, min body size multiple vs avg body to confirm displacement
    
    Returns:
        List of dicts with OB info
    """
    order_blocks = []
    df['candle_type'] = np.where(df['open'] > df['close'], -1, 1)
    # Calculate average body size for displacement check
    avg_body = (df['close'] - df['open']).abs().rolling(20).mean()
    
    for i in range(len(df) - lookahead):
        d, o, h, l, c = df.loc[i, ['date', 'open','high','low','close']]
        c_3 = df.loc[min(i+1+lookahead, len(df)-1), 'close']
        body = abs(c - o)
        
        # Bullish OB = last red candle before green drive
        if c < o:  # red candle
            forward_closes = df['close'].iloc[i+1:i+1+lookahead]
            forward_cdl_types = df['candle_type'].iloc[i+1:i+1+lookahead].sum()
            if all(forward_closes > c) and ((c_3 / c - 1)> displacement_factor) and (forward_cdl_types==lookahead):
                order_blocks.append({
                    "type": "bullish",
                    "date": d,
                    "index": i,
                    "zone_low": l,
                    "zone_high": h
                })
        
        # Bearish OB = last green candle before red drive
        if c > o:  # green candle
            forward_closes = df['close'].iloc[i+1:i+1+lookahead]
            forward_cdl_types = df['candle_type'].iloc[i+1:i+1+lookahead].sum()
            if all(forward_closes < c) and ((c / c_3 - 1) > displacement_factor) and (forward_cdl_types==-lookahead):
                order_blocks.append({
                    "type": "bearish",
                    "date": d,
                    "index": i,
                    "zone_low": l,
                    "zone_high": h
                })
    
    return order_blocks


In [3]:
def fibonacci_levels(high_price: float, low_price: float) -> dict:
    """
    Calculate Fibonacci retracement levels between two price points.
    
    Parameters:
        high_price (float): The swing high price.
        low_price (float): The swing low price.
    
    Returns:
        dict: Fibonacci levels with retracement percentages.
    """
    diff = high_price - low_price
    levels = {
        "0.0%": high_price,
        "23.6%": high_price - 0.236 * diff,
        "38.2%": high_price - 0.382 * diff,
        "50.0%": high_price - 0.5 * diff,
        "61.8%": high_price - 0.618 * diff,
        "78.6%": high_price - 0.786 * diff,
        "100.0%": low_price
    }
    return levels


In [4]:
fibonacci_levels(high_price=180.23, low_price=99.87) # Verified from trading view (SOLUSDT)

{'0.0%': 180.23,
 '23.6%': 161.26504,
 '38.2%': 149.53248,
 '50.0%': 140.05,
 '61.8%': 130.56752,
 '78.6%': 117.06703999999999,
 '100.0%': 99.87}

In [5]:
fibonacci_levels(high_price=203.59, low_price=297.95)

{'0.0%': 203.59,
 '23.6%': 225.85896,
 '38.2%': 239.63551999999999,
 '50.0%': 250.76999999999998,
 '61.8%': 261.90448,
 '78.6%': 277.75696,
 '100.0%': 297.95}

In [6]:
files = []
for f in os.listdir("data"):
    if f.endswith("SMC_merged.csv"):
        files.append(f)

files

['ABB_SMC_merged.csv',
 'ADANIENSOL_SMC_merged.csv',
 'ADANIENT_SMC_merged.csv',
 'ADANIGREEN_SMC_merged.csv',
 'ADANIPORTS_SMC_merged.csv',
 'ADANIPOWER_SMC_merged.csv',
 'AMBUJACEM_SMC_merged.csv',
 'APOLLOHOSP_SMC_merged.csv',
 'ASIANPAINT_SMC_merged.csv',
 'AXISBANK_SMC_merged.csv',
 'BAJAJ-AUTO_SMC_merged.csv',
 'BAJAJFINSV_SMC_merged.csv',
 'BAJAJHFL_SMC_merged.csv',
 'BAJAJHLDNG_SMC_merged.csv',
 'BAJFINANCE_SMC_merged.csv',
 'BANKBARODA_SMC_merged.csv',
 'BEL_SMC_merged.csv',
 'BHARTIARTL_SMC_merged.csv',
 'BOSCHLTD_SMC_merged.csv',
 'BPCL_SMC_merged.csv',
 'BRITANNIA_SMC_merged.csv',
 'CANBK_SMC_merged.csv',
 'CGPOWER_SMC_merged.csv',
 'CHOLAFIN_SMC_merged.csv',
 'CIPLA_SMC_merged.csv',
 'COALINDIA_SMC_merged.csv',
 'DABUR_SMC_merged.csv',
 'DIVISLAB_SMC_merged.csv',
 'DLF_SMC_merged.csv',
 'DMART_SMC_merged.csv',
 'DRREDDY_SMC_merged.csv',
 'EICHERMOT_SMC_merged.csv',
 'ETERNAL_SMC_merged.csv',
 'GAIL_SMC_merged.csv',
 'GODREJCP_SMC_merged.csv',
 'GRASIM_SMC_merged.csv',
 'HA

In [7]:
files[49]

'INDIGO_SMC_merged.csv'

In [8]:
df = pd.read_csv(os.path.join("data", files[49]))

In [9]:
# Ensure date is datetime and sorted for plotting
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df.sort_values("date", inplace=True)
    df.reset_index(drop=True, inplace=True)

bullish_pattern = [-1, 1, -1, 1]  # Bullish CoC+
bearish_pattern = [1, -1, 1, -1]  # Bearish CoC+

l = 20  # Lookback
df["CoC+"] = np.zeros(len(df))
for i in range(l, len(df)):
    # Use series to retain original indices for swings
    shl_series = df["shl_HighLow"].iloc[i - l : i].dropna()
    level_series = df["shl_Level"].iloc[i - l : i].dropna()
    # Use closes at the swing indices for CoC comparisons
    close_series = df.loc[shl_series.index, "close"]
    hist = shl_series.to_list()
    levels = level_series.to_list()
    closes = close_series.to_list()
    # Need at least 5 swings to evaluate the pattern logic safely
    if len(hist) < 5:
        continue
    if hist[-4:] == bullish_pattern:
        # Check Break of Structure with bearish expectancy and bullish CoC
        if (
            (levels[-5] > levels[-3])
            and (levels[-4] < levels[-2])
            and (levels[-3] < levels[-1])
        ):
            # Place marker at the second most recent swing's actual row index
            swing_index = shl_series.index[-2]
            df.loc[swing_index, "CoC+"] = 1
            df.loc[swing_index, "level"] = level_series.iloc[-2]
    if hist[-4:] == bearish_pattern:
        # Check Break of Structure with bullish expectancy and bearish CoC
        if (
            (levels[-5] < levels[-3])
            and (levels[-4] > levels[-2])
            and (levels[-3] > levels[-1])
        ):
            # Place marker at the second most recent swing's actual row index
            swing_index = shl_series.index[-2]
            df.loc[swing_index, "CoC+"] = -1
            df.loc[swing_index, "level"] = level_series.iloc[-2]

In [15]:
indices = df[df['CoC+']!=0].index
idx = indices[0]

In [21]:
type(idx)==np.int64, type(indices)==pd.core.indexes.base.Index

(True, True)

In [22]:
2*7*6

84

In [71]:
df.columns

Index(['index', 'symbol', 'date', 'open', 'high', 'low', 'close',
       'adjusted_close', 'volume', 'shl_HighLow', 'shl_Level', 'bos_BOS',
       'bos_CHOCH', 'bos_Level', 'bos_BrokenIndex', 'ob_OB', 'ob_Top',
       'ob_Bottom', 'ob_OBVolume', 'ob_MitigatedIndex', 'ob_Percentage',
       'fvg_FVG', 'fvg_Top', 'fvg_Bottom', 'fvg_MitigatedIndex',
       'liq_Liquidity', 'liq_Level', 'liq_End', 'liq_Swept', 'CoC+', 'level'],
      dtype='object')

In [72]:
OBs = detect_order_blocks(df, 3, displacement_factor=0.12)

In [73]:
df_ob = pd.DataFrame(OBs)
df_ob

,type,date,index,zone_low,zone_high
0,bearish,2020-02-17,7,1426.75,1493.40
1,bullish,2020-10-26,43,1253.70,1387.00
2,bearish,2022-02-07,110,1985.00,2282.10
3,bullish,2022-02-28,113,1634.00,1894.00
4,bearish,2022-04-18,120,1830.25,1951.00
5,bullish,2022-06-27,130,1571.75,1667.00
6,bullish,2022-11-07,149,1676.00,1817.95
7,bearish,2023-01-30,161,2060.00,2160.00
8,bullish,2023-04-10,171,1864.55,1922.95
9,bullish,2024-03-04,218,3026.00,3215.00


In [81]:
df['CoC+'].value_counts()

CoC+
 0.0    296
 1.0      1
-1.0      1
Name: count, dtype: int64

In [76]:
df.index = pd.to_datetime(df['date'])
df_ob.index = pd.to_datetime(df_ob['date'])

In [82]:
coc_ixd = df[df['CoC+']!=0].index
coc_ixd

DatetimeIndex(['2021-02-22', '2022-05-30'], dtype='datetime64[ns]', name='date', freq=None)

In [78]:
df.columns

Index(['index', 'symbol', 'date', 'open', 'high', 'low', 'close',
       'adjusted_close', 'volume', 'shl_HighLow', 'shl_Level', 'bos_BOS',
       'bos_CHOCH', 'bos_Level', 'bos_BrokenIndex', 'ob_OB', 'ob_Top',
       'ob_Bottom', 'ob_OBVolume', 'ob_MitigatedIndex', 'ob_Percentage',
       'fvg_FVG', 'fvg_Top', 'fvg_Bottom', 'fvg_MitigatedIndex',
       'liq_Liquidity', 'liq_Level', 'liq_End', 'liq_Swept', 'CoC+', 'level',
       'candle_type', 'ob_Type', 'ob_Low', 'ob_High'],
      dtype='object')

In [79]:
df['ob_Type'] = pd.Series()
df['ob_Low'] = pd.Series()
df['ob_High'] = pd.Series()
for i, row in df.iterrows():
    if row['CoC+']!=0:
        # Check if the date exists in df_ob before accessing
        if i in df_ob.index:
            OB = df_ob.loc[i]
            df['ob_Type'] = OB['type']
            df['ob_Low'] = OB['zone_low']
            df['ob_High'] = OB['zone_high']

In [80]:
for idx in coc_ixd:
    OB = df_ob.loc[idx:idx+pd.Timedelta(days=1)]
    if not OB.empty:
        print(len(OB))
        print("Found CoC + OB at", idx)
        print(OB['zone_high'].iloc[0], OB['zone_low'].iloc[0])
        print("OB initiaiton: ", OB['date'].iloc[0])

In [196]:
import json
with open(os.path.join("data", "ICICIBANK_Hourly.json"), "r") as f:
    res = json.load(f)

In [197]:
df_h = pd.DataFrame(res)
df_h = df_h.drop(['timestamp', 'gmtoffset'], axis=1)

In [198]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])

In [199]:
df_h.index = df_h['datetime']

In [200]:
df_h = df_h.resample("4h").agg(
    {"open": "first", "high": "max", "low": "min", "close": "last"}
)

df_h

,open,high,low,close
datetime,,,,
2020-10-12 00:00:00,403.450012,410.000000,403.450012,407.000000
2020-10-12 04:00:00,406.049987,407.899993,400.700012,404.049987
2020-10-12 08:00:00,404.299987,406.350006,401.750000,403.500000
2020-10-12 12:00:00,NaN,NaN,NaN,NaN
2020-10-12 16:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2025-09-18 16:00:00,NaN,NaN,NaN,NaN
2025-09-18 20:00:00,NaN,NaN,NaN,NaN
2025-09-19 00:00:00,1417.300048,1417.300048,1403.400024,1405.199951


In [202]:
df_h.dropna().to_csv("example.csv", index=False)

In [204]:
list(df[df['CoC+']!=0].index)

[126, 193, 255]

In [214]:
OBs = pd.DataFrame(OBs)
OBs[OBs['index']==18].empty

True

In [215]:
import pandas as pd


def wick_rejection(candle, threshold=0.6):
    """
    Detect strong lower wick rejection.
    threshold = portion of wick relative to full candle.
    """
    o, h, l, c = candle
    body_low = min(o, c)
    lower_wick = body_low - l
    full_range = h - l
    if full_range == 0:
        return False
    return (lower_wick / full_range) >= threshold


def playbook_long(df_weekly: pd.DataFrame, df_4h: pd.DataFrame):
    """
    Full pipeline for bullish setups.
    """
    signals = []

    # 1. Detect CHoCH+
    choch_indices = list(df[df['CoC+']!=0].index)
    OBs = detect_order_blocks(df_weekly)
    OBs = pd.DataFrame(OBs)
    for idx in choch_indices:
        # 2. Find Weekly OB
        if OBs[OBs['index']==idx].empty:
            continue
        ob = OBs[OBs['index']==idx].to_dict()

        # 3. Draw Fib from OB low → swing high (3rd green candle after OB)
        swing_high = df_weekly.loc[idx + 3, "high"]
        fib = fibonacci_levels(high_price=swing_high, low_price=ob["low"])

        fib_618 = fib["0.618"]

        # 4. Drop to 4H timeframe
        df_zone = df_4h[df_4h["low"] <= fib_618]  # candles touching fib 618

        tapped = False
        for i, row in df_zone.iterrows():
            candle = (row.open, row.high, row.low, row.close)

            if not tapped:
                # first touch = confirmation
                tapped = True
            else:
                # 2nd touch → entry if wick rejection
                if wick_rejection(candle, threshold=0.6):
                    signals.append({
                        "time": row.name,
                        "entry_price": row.close,
                        "stop_loss": ob["low"],
                        "take_profit": swing_high
                    })
    return signals


In [216]:
playbook_long(df, df_h)

[]

In [2]:
import pandas as pd
import os

In [8]:
nifty_100 = pd.read_csv("ind_nifty100list.csv")
data = pd.DataFrame()
for _, row in nifty_100.iterrows():
    ticker = row['Symbol']
    df = pd.read_csv(os.path.join("plots_data", f"{ticker}.csv")).drop('index', axis=1)
    choch_indices = list(df[df['CoC+']==1].index)
    for idx in choch_indices:
        slice = df.loc[idx:idx+5]
        if any(slice['fvg_FVG']==1):
            data = pd.concat([data, slice])

In [12]:
data.to_csv("CoCPlus_FVG.csv", index=False)

In [4]:
df = pd.read_csv("all_signals_v3.csv")

In [58]:
df = df.drop_duplicates(subset='take_profit')

In [35]:
df["return"] = df["return"].fillna(df["returns"])
df = df.drop(columns=["returns"])

In [13]:
df[5:].loc[5+1: 5+5, 'entry_price']

6       562.450012
7      4232.350097
8      4205.000000
9      3846.000000
10    11051.000000
Name: entry_price, dtype: float64

In [55]:
df['returns'].sum()

3.598218910776225

In [60]:
df.to_csv('all_signals_long_short.csv', index=False)

In [ ]:
time, entry = df.iloc[0][['time', 'entry_price']]

In [69]:
time, entry

('2021-04-05 04:00:00', 1080.0)

In [77]:
df.loc[0]

setup_type                        LONG
time               2021-04-05 04:00:00
entry_price                     1080.0
stop_loss                  1017.799987
take_profit                     1150.1
outcome_6                           SL
outcome_8                           SL
initiation_date             2024-05-06
returns                      -0.057593
exit_time          2021-04-08 00:00:00
ticker                      ADANIENSOL
Name: 0, dtype: object

In [15]:
from order_block_detector import load_and_detect_order_blocks

In [16]:
results, df, detector = load_and_detect_order_blocks('ohlc_weekly_data/AMBUJACEM_weekly.csv')

Detected 0 active bullish order blocks
Detected 1 active bearish order blocks
Total bullish signals: 0
Total bearish signals: 22
Total mitigation signals: 673


In [17]:
results

{'bullish_order_blocks': [],
 'bearish_order_blocks': [OrderBlock(left_time=Timestamp('2025-07-28 00:00:00'), top=624.8, bottom=606.775, avg=615.7874999999999, ob_type='bearish', is_mitigated=False, mitigation_time=None)],
 'bull_signals': [],
 'bear_signals': [{'date': Timestamp('2020-04-20 00:00:00'),
   'price': 184.05,
   'type': 'bearish_ob_formed'},
  {'date': Timestamp('2020-08-31 00:00:00'),
   'price': 225.5,
   'type': 'bearish_ob_formed'},
  {'date': Timestamp('2020-11-23 00:00:00'),
   'price': 256.0,
   'type': 'bearish_ob_formed'},
  {'date': Timestamp('2021-01-25 00:00:00'),
   'price': 260.9,
   'type': 'bearish_ob_formed'},
  {'date': Timestamp('2021-03-22 00:00:00'),
   'price': 290.5,
   'type': 'bearish_ob_formed'},
  {'date': Timestamp('2021-08-23 00:00:00'),
   'price': 409.5,
   'type': 'bearish_ob_formed'},
  {'date': Timestamp('2021-11-22 00:00:00'),
   'price': 418.0,
   'type': 'bearish_ob_formed'},
  {'date': Timestamp('2022-03-28 00:00:00'),
   'price': 341

In [18]:
import plotly.graph_objects as go
import pandas as pd
from typing import Optional, Dict
def plot_order_blocks_plotly(df: pd.DataFrame, results: Dict,
                             start_date: Optional[str] = None,
                             end_date: Optional[str] = None):
    """
    Plot full order block detection history using Plotly
    """
    plot_df = df.copy()
    if start_date:
        plot_df = plot_df[plot_df.index >= pd.to_datetime(start_date)]
    if end_date:
        plot_df = plot_df[plot_df.index <= pd.to_datetime(end_date)]
    
    fig = go.Figure()

    # --- Candlesticks ---
    fig.add_trace(go.Candlestick(
        x=plot_df.index,
        open=plot_df['open'],
        high=plot_df['high'],
        low=plot_df['low'],
        close=plot_df['close'],
        name='Candles'
    ))

    # --- Order Blocks (All, not only active) ---
    for ob in results['all_bullish_obs']:
        color = "rgba(0, 200, 0, 0.2)" if not ob.is_mitigated else "rgba(0, 200, 0, 0.05)"
        end_time = plot_df.index[-1] if ob.mitigation_time is None else ob.mitigation_time
        fig.add_shape(type="rect",
            x0=ob.left_time, x1=end_time,
            y0=ob.bottom, y1=ob.top,
            line=dict(color="green"),
            fillcolor=color,
            layer="below"
        )
        # average line
        fig.add_shape(type="line",
            x0=ob.left_time, x1=end_time,
            y0=ob.avg, y1=ob.avg,
            line=dict(color="darkgreen", dash="dash"),
            layer="above"
        )

    for ob in results['all_bearish_obs']:
        color = "rgba(200, 0, 0, 0.2)" if not ob.is_mitigated else "rgba(200, 0, 0, 0.05)"
        end_time = plot_df.index[-1] if ob.mitigation_time is None else ob.mitigation_time
        fig.add_shape(type="rect",
            x0=ob.left_time, x1=end_time,
            y0=ob.bottom, y1=ob.top,
            line=dict(color="red"),
            fillcolor=color,
            layer="below"
        )
        fig.add_shape(type="line",
            x0=ob.left_time, x1=end_time,
            y0=ob.avg, y1=ob.avg,
            line=dict(color="darkred", dash="dash"),
            layer="above"
        )

    # --- Formation signals ---
    bull_form = results['bull_signals']
    bear_form = results['bear_signals']
    mitigs = results['mitigation_signals']

    if bull_form:
        fig.add_trace(go.Scatter(
            x=[s['date'] for s in bull_form],
            y=[s['price'] for s in bull_form],
            mode="markers",
            marker=dict(color="green", symbol="triangle-up", size=12),
            name="Bullish OB Formed"
        ))

    if bear_form:
        fig.add_trace(go.Scatter(
            x=[s['date'] for s in bear_form],
            y=[s['price'] for s in bear_form],
            mode="markers",
            marker=dict(color="red", symbol="triangle-down", size=12),
            name="Bearish OB Formed"
        ))

    if mitigs:
        fig.add_trace(go.Scatter(
            x=[s['date'] for s in mitigs],
            y=[s['price'] for s in mitigs],
            mode="markers",
            marker=dict(color="blue", symbol="x", size=10),
            name="OB Mitigated"
        ))

    # --- Layout ---
    fig.update_layout(
        title="Full Order Block Detection History",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig.show()


In [19]:
plot_order_blocks_plotly(df, results)

In [20]:
import plotly.graph_objects as go

def plot_order_block_zones(df: pd.DataFrame, results: Dict,
                           start_date: Optional[str] = None,
                           end_date: Optional[str] = None):
    """
    Plot OB zones with Plotly (zones stop at mitigation or extend to chart end)
    """
    plot_df = df.copy()
    if start_date:
        plot_df = plot_df[plot_df.index >= pd.to_datetime(start_date)]
    if end_date:
        plot_df = plot_df[plot_df.index <= pd.to_datetime(end_date)]

    fig = go.Figure()

    # --- Candles ---
    fig.add_trace(go.Candlestick(
        x=plot_df.index,
        open=plot_df['open'],
        high=plot_df['high'],
        low=plot_df['low'],
        close=plot_df['close'],
        name="Candles"
    ))

    last_time = plot_df.index[-1]

    # --- Bullish OB Zones ---
    for ob in results["all_bullish_obs"]:
        end_time = ob.mitigation_time if ob.mitigation_time else last_time
        color = "rgba(0, 200, 0, 0.25)" if not ob.is_mitigated else "rgba(0, 200, 0, 0.08)"

        fig.add_shape(
            type="rect",
            x0=ob.left_time,
            x1=end_time,
            y0=ob.bottom,
            y1=ob.top,
            fillcolor=color,
            line=dict(color="green"),
            layer="below"
        )

    # --- Bearish OB Zones ---
    for ob in results["all_bearish_obs"]:
        end_time = ob.mitigation_time if ob.mitigation_time else last_time
        color = "rgba(200, 0, 0, 0.25)" if not ob.is_mitigated else "rgba(200, 0, 0, 0.08)"

        fig.add_shape(
            type="rect",
            x0=ob.left_time,
            x1=end_time,
            y0=ob.bottom,
            y1=ob.top,
            fillcolor=color,
            line=dict(color="red"),
            layer="below"
        )

    # --- OB Form signals ---
    if results["bull_signals"]:
        fig.add_trace(go.Scatter(
            x=[s["date"] for s in results["bull_signals"]],
            y=[s["price"] for s in results["bull_signals"]],
            mode="markers",
            marker=dict(color="green", symbol="triangle-up", size=12),
            name="Bull OB Formed"
        ))

    if results["bear_signals"]:
        fig.add_trace(go.Scatter(
            x=[s["date"] for s in results["bear_signals"]],
            y=[s["price"] for s in results["bear_signals"]],
            mode="markers",
            marker=dict(color="red", symbol="triangle-down", size=12),
            name="Bear OB Formed"
        ))

    # --- Mitigation markers ---
    if results["mitigation_signals"]:
        fig.add_trace(go.Scatter(
            x=[s["date"] for s in results["mitigation_signals"]],
            y=[s["price"] for s in results["mitigation_signals"]],
            mode="markers",
            marker=dict(color="blue", symbol="x", size=10),
            name="Mitigated"
        ))

    # --- Layout ---
    fig.update_layout(
        title="Order Block Zones (Bullish / Bearish)",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig
    # fig.show()


In [21]:
fig = plot_order_block_zones(df, results)

In [22]:
fig.write_html('lux_algo_ob.html')